In [1]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd

data_folder = "./ml-20m"

ratings_df = pd.read_csv(data_folder + "/ratings.csv")
movies_df = pd.read_csv(data_folder + "/movies.csv")

ratings_df = ratings_df.rename(columns={'userId': 'user', 'movieId': 'item'})
movies_df = movies_df.rename(columns={'movieId': 'item', 'title': 'title_original'})

In [2]:
def compute_title_year(row):
    # read the title of the specified row, and cast it to the str type,
    # then use the strip() method to eliminate the spaces from the beginning and the end of the string
    title_original = str(row['title_original']).strip()

    # select the substring from the fifth-to-last (included) character to the last (excluded)
    year = title_original[-5:-1]

    # use the isdigit method to determine if the substring selected is a number
    if year.isdigit():
        # remove the year from the title
        # select the substring from the beninning to the sixth-to-last (excluded)
        # then use the strip() method to remove spaces at the beginning and at the end of the string
        # finally, use the lower() method to obtain the title in lower case
        title = title_original[:-6].strip().lower()

        # return the year and the title
        return int(year), title
    else:
        # nothing to do, return 0 as year and the title
        return 0, title_original.lower()


movies_df[['year', 'title']] = movies_df.apply(compute_title_year, axis=1, result_type='expand')
movies_df = movies_df[['item', 'title', 'year', 'genres']]
display(movies_df)

,item,title,year,genres
0,1,toy story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,2,jumanji,1995,Adventure|Children|Fantasy
2,3,grumpier old men,1995,Comedy|Romance
3,4,waiting to exhale,1995,Comedy|Drama|Romance
4,5,father of the bride part ii,1995,Comedy
...,...,...,...,...
27273,131254,kein bund für's leben,2007,Comedy
27274,131256,"feuer, eis & dosenbier",2002,Comedy
27275,131258,the pirates,2014,Adventure
27276,131260,rentun ruusu,2001,(no genres listed)


In [3]:
movies_df = movies_df.drop_duplicates(subset='title', keep='first')

# Join the ratings to the genres
ratings_genres_df = pd.merge(movies_df, ratings_df,  how='inner', left_on=['item'], right_on = ['item'])
ratings_genres_df = ratings_genres_df[['item', 'user', 'rating', 'title', 'genres']]

# keep only the user and genres
user_genres = ratings_genres_df.loc[:, ['user', 'genres']]

# Create a user only dataframe
user_df = user_genres.drop_duplicates(subset=['user'], keep='first')
user_df = user_df['user']
user_df = pd.DataFrame(user_df)
user_df = user_df.set_index('user')
user_df = user_df.sort_index(ascending=True)

user_genres = user_genres.set_index('user')
user_genres = user_genres.sort_values(by="user", ascending=True)

display(user_genres)
display(user_df)

,genres
user,
1,Adventure|Animation|Fantasy|Sci-Fi
1,Action|Adventure
1,Action|Western
1,Drama|Western
1,Adventure|Comedy|Fantasy
...,...
138493,Comedy
138493,Adventure|Animation|Children|Comedy
138493,Adventure|Animation|Children|Comedy


""
user
1
2
3
4
5
...
138489
138490
138491


In [4]:
import numpy as np

def count_rated_genres(user_id, df):

    user_genre = df.loc[user_id,:]
    rated_genres = set()

    for genre in user_genre.genres:
        rated_genres.update(genre.split('|'))

    return len(rated_genres)

user_genres_rated = []
for user in user_df.index:
     user_genres_rated.append(count_rated_genres(user, user_genres))


In [5]:
user_genres_rated_df = pd.DataFrame({
    'user' : user_df.index.tolist(),
    'genres_rated' : user_genres_rated
})

display(user_genres_rated_df)

mean_genres_rated = np.mean(user_genres_rated)
print(mean_genres_rated)
user_genres_rated_df = user_genres_rated_df[user_genres_rated_df['genres_rated'] > mean_genres_rated]

display(user_genres_rated_df)

,user,genres_rated
0,1,17
1,2,18
2,3,18
3,4,15
4,5,17
...,...,...
138488,138489,17
138489,138490,18
138490,138491,14
138491,138492,17


16.360133725170225


,user,genres_rated
0,1,17
1,2,18
2,3,18
4,5,17
6,7,18
...,...,...
138486,138487,17
138488,138489,17
138489,138490,18
138491,138492,17


In [6]:
count_ratings = ratings_df.groupby(['user']).count()
selected = count_ratings['rating'] > count_ratings['rating'].mean()

expert_users = count_ratings.loc[selected]
expert_users = pd.DataFrame(expert_users)
expert_users = expert_users.reset_index()
expert_users = expert_users.merge(user_genres_rated_df, how='inner', left_on='user', right_on='user')
expert_users = expert_users.drop(['timestamp', 'item'], axis=1)
expert_users.rename(columns={'rating' : 'num_ratings'}, inplace=True)

display(expert_users)

,user,num_ratings,genres_rated
0,1,175,17
1,3,187,18
2,7,276,18
3,11,504,19
4,14,243,17
...,...,...,...
35870,138483,276,19
35871,138484,148,18
35872,138486,193,18
35873,138490,151,18


In [7]:

random_user = expert_users.sample()
random_user = random_user['user']
random_user_ratings = pd.merge(random_user, ratings_genres_df, how='left', left_on='user', right_on='user')

display(random_user_ratings)

,user,item,rating,title,genres
0,27469,1,4.0,toy story,Adventure|Animation|Children|Comedy|Fantasy
1,27469,2,2.5,jumanji,Adventure|Children|Fantasy
2,27469,5,2.0,father of the bride part ii,Comedy
3,27469,6,4.5,heat,Action|Crime|Thriller
4,27469,11,2.5,"american president, the",Comedy|Drama|Romance
...,...,...,...,...,...
3054,27469,112623,3.5,dawn of the planet of the apes,Sci-Fi
3055,27469,112852,4.0,guardians of the galaxy,Action|Adventure|Sci-Fi
3056,27469,114704,2.5,space station 76,Comedy|Drama|Sci-Fi
3057,27469,117109,5.0,too many cooks,Comedy


In [8]:

genre_set = set()
for genres in ratings_genres_df.genres:
    genre_set.update(genres.split('|'))


In [9]:
genre_df = ratings_genres_df[['item', 'genres']]
genre_df = pd.DataFrame(genre_df)

i = 2
for genre in genre_set:
    genre_df.insert(i, genre, False)
    i += 1

genre_df = genre_df.drop_duplicates(subset='item', keep='first')
genre_df = genre_df.set_index('item')

display(genre_df)

,genres,(no genres listed),Musical,Drama,Thriller,Children,Action,Animation,Film-Noir,Romance,...,Crime,Comedy,Fantasy,Horror,Sci-Fi,Documentary,IMAX,Adventure,War,Western
item,,,,,,,,,,,,,,,,,,,,,
1,Adventure|Animation|Children|Comedy|Fantasy,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Adventure|Children|Fantasy,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Comedy|Romance,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Comedy|Drama|Romance,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,Comedy,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131252,Comedy|Horror,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131254,Comedy,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131256,Comedy,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:

for item in genre_df.index:
    genres = genre_df.loc[item, 'genres']
    genre_list = genres.split('|')
    for genre in genre_list:
        genre_df.at[item, genre] = True

genre_df = genre_df.drop(['genres'], axis=1)

display(genre_df)

,(no genres listed),Musical,Drama,Thriller,Children,Action,Animation,Film-Noir,Romance,Mystery,Crime,Comedy,Fantasy,Horror,Sci-Fi,Documentary,IMAX,Adventure,War,Western
item,,,,,,,,,,,,,,,,,,,,
1,False,False,False,False,True,False,True,False,False,False,False,True,True,False,False,False,False,True,False,False
2,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131252,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False
131254,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
131256,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [13]:

def find_movies_with_genres(find_genres, df):
    for genre in find_genres:
        df = df.loc[(df[genre] == True)]

    return df


display(find_movies_with_genres(['Crime', 'Drama'], genre_df))

,(no genres listed),Musical,Drama,Thriller,Children,Action,Animation,Film-Noir,Romance,Mystery,Crime,Comedy,Fantasy,Horror,Sci-Fi,Documentary,IMAX,Adventure,War,Western
item,,,,,,,,,,,,,,,,,,,,
16,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
20,False,False,True,True,False,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False
22,False,False,True,True,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False
30,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
36,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129937,False,False,True,True,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False
130083,False,False,True,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
130219,False,False,True,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False


In [32]:
tags_df = pd.DataFrame(pd.read_csv(data_folder + "/tags.csv"))
tags_df = tags_df.rename(columns={'movieId': 'item'})
tags_df = tags_df.drop(['timestamp', 'userId'], axis=1)
tags_df = tags_df.sort_values(by="item", ascending=True)
tags_df = tags_df.set_index('item')
# tags_df.index = np.arrange(len(tags_df.index))
display(tags_df)

,tag
item,
1,friendship
1,animation
1,animated
1,adventure
1,animation
...,...
131258,Korea
131258,bandits
131258,whale


In [39]:
tags_set = set()
for tag in tags_df.tag:
    tags_set.add(repr(tag).strip().lower().replace('\'', ''))

print(tags_set)

{'what are you stuffed with?', 'had a hard time following the plot of the movie.', 'lack of imagination', 'imax dmr reissue in 2014', 'kungfu hackers', 'ghost children', 'ellen muth', 'see: the spongebob squarepants movie', 'bear necessities', 'omar sy', 'lame score', 'scott cooper', 'dinosaurs', 'remakeof:the time machine(1960)', 'noise in space', 'santiago mitre', 'trailer was better than the movie', 'male', 'tacky', 'beer: the breakfast of champions', 'gag:tossing lighter out of car', 'victor buono', 'brain injury', 'no backstory', 'lucy liu', 'dumbed down', 'author:brian aldiss', 'borneo', 'azita rayeji', 'gábor herendi', 'robert sheehan', 'waste of celluloid', 'documentary footage', 'ágnes hranitzky', 'nyctophobia', 'charles walters', '"doesnt show the fights"', 'scott porter', 'repulsive', 'pedro costa', 'lolita', 'misbegotten', 'lousy acting', 'animal:shark', 'parrots', 'tuvan throat singing', 'sequel:worse', 'socialism', 'prehensile hair', 'jarvier bardem', 'disgust', 'signatur